# LAR integrals

* ### Paolo Di Simone 584638
* ### Filippo Gaglioti 582704
* ### Federico Pasqui 501749

## API:
* *function M(alpha::Int, beta::Int)::Float64*
* *function TT(tau::Array{Float64,2}, alpha::Int, beta::Int, gamma::Int, signedInt::Bool=false)*
* *function II(P::LAR, alpha::Int, beta::Int, gamma::Int, signedInt=false)::Float64*
* *function III(P::LAR, alpha::Int, beta::Int, gamma::Int)::Float64*
* *function surface(P::LAR, signedInt::Bool=false)::Float64*
* *function volume(P::LAR)::Float64*
* *function firstMoment(P::LAR)::Array{Float64,1}*
* *function secondMoment(P::LAR)::Array{Float64,1}*
* *function inertiaProduct(P::LAR)::Array{Float64,1}*
* *function centroid(P::LAR)::Array{Float64,1}*
* *function inertiaMoment(P::LAR)::Array{Float64,1}*
* *function chainAreas(V::Array{Float64,2},EV::Array{Int64,2},chains::Array{Int64,2})* (**non sviluppata in questo notebook**)
* *function chainAreas(V::Array{Float64,2}, EV::Array{Int64,2}, chains::Array{Array{Int64,1},1})* (**non sviluppata in questo notebook**)

## Librerie importate e costanti

In [ ]:
using LinearAlgebra

In [ ]:
const Points = Matrix
const Cells = Array{Array{Int,1},1}
const LAR = Union{ Tuple{Points, Cells},Tuple{Points, Cells, Cells} }

## Codice base

Le seguenti funzioni, rispetto al codice base, presentano solo delle piccole modifiche che sono state necessarie ai fini dell'analisi e della conseguente ottimizzazione/parallelizzazione del codice.

In [ ]:
function M(alpha::Int, beta::Int)::Float64
    a = 0
    for l=0:(alpha + 1)
        a += binomial(alpha+1,l) * (-1)^l/(l+beta+1)
    end
    return a/(alpha + 1)
end

La funzione `TT(tau::Array{Float64,2}, alpha::Int, beta::Int, gamma::Int, signedInt::Bool=false)` è stata divisa in funzioni più piccole che sono: `s1(a, b, alpha, beta, gamma, vo)`, `s2(a, b, h, k, m)`, `s3(a, b, h, k, m, i)` ed `s4(a, b, h, k, m, i, j)`.

In [ ]:
function s4(a, b, h, k, m, i, j)
    ss4 = 0.0
    for l=0:m
        ss4 += binomial(m,l) * a[3]^(m-l) * b[3]^l * M(h+k+m-i-j-l, i+j+l)
    end
    return ss4
end

In [ ]:
function s3(a, b, h, k, m, i)
    ss3 = 0.0
    for j=0:k
        ss3 += binomial(k,j) * a[2]^(k-j) * b[2]^j * s4(a, b, h, k, m, i, j)
    end
    return ss3
end

In [ ]:
function s2(a, b, h, k, m)
    ss2 = 0.0
    for i=0:h 
        ss2 += binomial(h,i) * a[1]^(h-i) * b[1]^i * s3(a, b, h, k, m, i);
    end
    return ss2
end

In [ ]:
function s1(a, b, alpha, beta, gamma, vo)
    ss1 = 0.0
    for x=0:((alpha+1) * (beta+1) * (gamma+1))
        h = x ÷ ((beta+1) * (gamma+1))
        k = (x - h * (beta+1) * (gamma+1)) ÷ (gamma + 1) 
        m = (x - h * (beta+1) * (gamma+1)) % (gamma + 1) 
        ss1 += binomial(alpha,h) * binomial(beta,k) * binomial(gamma,m) * vo[1]^(alpha-h) * vo[2]^(beta-k) * vo[3]^(gamma-m) * s2(a, b, h, k, m)
    end
    return ss1
end

In [ ]:
function TT(tau::Array{Float64,2}, alpha::Int, beta::Int, gamma::Int, signedInt::Bool=false)
    vo,va,vb = tau[:,1],tau[:,2],tau[:,3]
    a = va - vo
    b = vb - vo
    c = cross(a,b)
    if signedInt == true
        return s1(a, b, alpha, beta, gamma, vo) * norm(c) * sign(c[3])
    else
        return s1(a, b, alpha, beta, gamma, vo) * norm(c)
    end
end

In [ ]:
function II(P::LAR, alpha::Int, beta::Int, gamma::Int, signedInt=false)::Float64
    w = 0
    V, FV = P
    if typeof(FV) == Array{Int64,2}
        FV = [FV[:,k] for k=1:size(FV,2)]
    end
    for i=1:length(FV)
        tau = hcat([V[:,v] for v in FV[i]]...)
        if size(tau,2) == 3
            term = TT(tau, alpha, beta, gamma, signedInt)
            if signedInt
                w += term
            else
                w += abs(term)
            end
        elseif size(tau,2) > 3
            println("ERROR: FV[$(i)] is not a triangle")
        else
            println("ERROR: FV[$(i)] is degenerate")
        end
    end    
    return w
end

Le funzioni `III(P::LAR, alpha::Int, beta::Int, gamma::Int, signedInt::Bool=false)::Float64` `volume(P::LAR, signedInt::Bool=false)::Float64` sono state leggermente modificate, aggiungendo il parametro `signedInt::Bool=false`.

In [ ]:
function III(P::LAR, alpha::Int, beta::Int, gamma::Int, signedInt::Bool=false)::Float64
    w = 0
    V, FV = P
    for i=1:length(FV)
        tau = hcat([V[:,v] for v in FV[i]]...)
        vo,va,vb = tau[:,1],tau[:,2],tau[:,3]
        a = va - vo
        b = vb - vo
        c = cross(a,b)
        term = c[1]/norm(c) * TT(tau, alpha+1, beta, gamma, signedInt)
        if signedInt
            w += term
        else
            w += abs(term)
        end
    end
    return w/(alpha + 1)
end

In [ ]:
function III(P::LAR, alpha::Int, beta::Int, gamma::Int, signedInt::Bool=false)::Float64
    w = 0
    V, FV = P
    array = zeros(length(FV))
    @threads for i=1:length(FV)
        tau = hcat([V[:,v] for v in FV[i]]...)
        vo,va,vb = tau[:,1],tau[:,2],tau[:,3]
        a = va - vo
        b = vb - vo
        c = cross(a,b)
        term = c[1]/norm(c) * TT(tau, alpha+1, beta, gamma, signedInt)
        if signedInt
            @inbounds array[i] = term
        else
            @inbounds array[i] = abs(term)
        end
    end
    w = sum(array)
    return w/(alpha + 1)
end

In [ ]:
function surface(P::LAR, signedInt::Bool=false)::Float64
    return II(P, 0, 0, 0, signedInt)
end

In [ ]:
function volume(P::LAR, signedInt::Bool=false)::Float64
    return III(P, 0, 0, 0, signedInt)
end

In [ ]:
function firstMoment(P::LAR)::Array{Float64,1}
    out = zeros(3)
    out[1] = III(P, 1, 0, 0)
    out[2] = III(P, 0, 1, 0)
    out[3] = III(P, 0, 0, 1)
    return out
end

In [ ]:
function secondMoment(P::LAR)::Array{Float64,1}
    out = zeros(3)
    out[1] = III(P, 2, 0, 0)
    out[2] = III(P, 0, 2, 0)
    out[3] = III(P, 0, 0, 2)
    return out
end

In [ ]:
function inertiaProduct(P::LAR)::Array{Float64,1}
    out = zeros(3)
    out[1] = III(P, 0, 1, 1)
    out[2] = III(P, 1, 0, 1)
    out[3] = III(P, 1, 1, 0)
    return out
end

In [ ]:
function centroid(P::LAR)::Array{Float64,1}
    return firstMoment(P)./volume(P)
end

In [ ]:
function inertiaMoment(P::LAR)::Array{Float64,1}
    out = zeros(3)
    result = secondMoment(P)
    out[1] = result[2] + result[3]
    out[2] = result[3] + result[1]
    out[3] = result[1] + result[2]
    return out
end

## Esempi

Nella seguente sezione è riportato l'esempio utilizzato per valutare le prestazioni del codice.

In [ ]:
using LinearAlgebraicRepresentation
using Plasm
using BenchmarkTools
Lar = LinearAlgebraicRepresentation

Di seguito si riporta la funzione contenuta nel modulo https://github.com/cvdlab/LinearAlgebraicRepresentation.jl/blob/master/src/utilities.jl dal momento che c'era un errore nel codice e quindi l'oggetto restituito non era corretto.

In [ ]:
function obj2lar(path)
    vs = Array{Float64, 2}(undef, 0, 3)
    edges = Array{Array{Array{Int, 1}, 1}, 1}()
    faces = Array{Array{Array{Int, 1}, 1}, 1}()
    push!(edges, Array{Array{Int, 1}, 1}[])
    push!(faces, Array{Array{Int, 1}, 1}[])
    g = 1

    open(path, "r") do fd
        for line in eachline(fd)
            elems = split(line)
            if length(elems) > 0
                if elems[1] == "v"
                    
                    x = parse(Float64, elems[2])
                    y = parse(Float64, elems[3])
                    z = parse(Float64, elems[4])

                    vs = [vs; x y z]
                    
                elseif elems[1] == "f"  
                    
                    v1 = parse(Int, split(elems[2], "/")[1])
                    v2 = parse(Int, split(elems[3], "/")[1])
                    v3 = parse(Int, split(elems[4], "/")[1])
                    
                    e1 = sort([v1, v2])
                    e2 = sort([v2, v3])
                    e3 = sort([v3, v1])
                    

                    push!(edges[g], e1)
                    push!(edges[g], e2)
                    push!(edges[g], e3)

                    push!(faces[g], sort([v1, v2, v3]))

                elseif elems[1] == "g"

                    g += 1
                    push!(edges, Array{Array{Int, 1}, 1}[])
                    push!(faces, Array{Array{Int, 1}, 1}[])
                    
                end
            end
        end
    end

    return convert(Lar.Points, vs'), edges[1:end], faces[1:end]
end

Esempio calcolo del volume del modello 3D *Stanford Bunny* (file bunny.obj)

In [ ]:
V, EV, FV = obj2lar("stanford-bunny.obj")
EV = EV[1]
FV = FV[1]
VV = [[k] for k=1:size(V,2)]
model = (V, (VV, EV, FV))
P = V, FV

In [ ]:
Plasm.view(V, FV)

In [ ]:
volume(P)

In [ ]:
@benchmark volume(P)

## Ottimizzazione

In questa sezione è riportato lo studio relativo alla funione `TT(tau::Array{Float64,2}, alpha::Int, beta::Int, gamma::Int, signedInt::Bool=false)` con conseguente ottimizzazione ottenuta "scompattandola" in funzione dei paramentri $\alpha, \beta, \gamma$. 

Per quanto riguarda il calcolo della superficie del singolo triangolo, si può notare che quando alla funzione `s1(a, b, alpha, beta, gamma, vo)` vengono passati i parametri `alpha = 0, beta = 0, gamma = 0` essa restituisce come valore `M(0,0)`; questo accade quando si calcola la superficie di una figura con la funzione `surface(P::LAR, signedInt::Bool=false)::Float64`. Allo stesso modo, quando i parametri sono `alpha = 1, beta = 0, gamma = 0` essa restituisce come valore l'espressione `vo[1] * M(0,0) + a[1] * M(1,0) + b[1] * M(0,1)`; questo accade quando si calcola il volume con la funzione `volume(P::LAR, signedInt::Bool=false)::Float64`. Quindi, tenendo conto di questi due casi particolari la funzione `TT(tau::Array{Float64,2}, alpha::Int, beta::Int, gamma::Int, signedInt::Bool=false)` può essere scompattata come segue.

Nella relazione sono riportati tutti i casi analizzati, quindi per il calcolo dei momenti e dei prodotti d'inerzia (oltre che superficie e volume).

In [ ]:
function TT(tau::Array{Float64,2}, alpha::Int, beta::Int, gamma::Int, signedInt::Bool=false)
    vo,va,vb = tau[:,1],tau[:,2],tau[:,3]
    a = va - vo
    b = vb - vo
    c = cross(a,b)
    if alpha == 0 && beta == 0 && gamma == 0 #surface
        area_tt = M(0,0)
    elseif alpha == 1 && beta == 0 && gamma == 0 #volume
        area_tt = vo[1]*M(0,0) + a[1]*M(1,0) + b[1]*M(0,1)
    elseif alpha == 2 && beta == 0 && gamma == 0 #firstMoment
        area_tt = vo[1]^2*M(0,0) + 
                  2*vo[1]*(a[1]*M(1,0) + b[1]*M(0,1)) + 
                  a[1]^2*M(2,0) + 2*a[1]*b[1]*M(1,1) + b[1]^2*M(0,2)
    elseif alpha == 1 && beta == 1 && gamma == 0 #firstMoment
        area_tt = vo[1]*vo[2]*M(0,0) + 
                  vo[1]*(a[2]*M(1,0) + b[2]*M(0,1)) + 
                  vo[2]*(a[1]*M(1,0) + b[1]*M(0,1)) + 
                  a[1]*(a[2]*M(2,0) + b[2]*M(1,1)) + b[1]*(a[2]*M(1,1) + b[2]*M(0,2))
    elseif alpha == 1 && beta == 0 && gamma == 1 #firstMoment
        area_tt = vo[1]*vo[3]*M(0,0) + 
                  vo[1]*(a[3]*M(1,0) + b[3]*M(0,1)) + 
                  vo[3]*(a[1]*M(1,0) + b[1]*M(0,1)) + 
                  a[1]*(a[3]*M(2,0) + b[3]*M(1,1)) + b[1]*(a[3]*M(1,1) + b[3]*M(0,2))
    elseif alpha == 3 && beta == 0 && gamma == 0 #secondMoment
        area_tt = vo[1]^3*M(0,0) + 
                  3*vo[1]^2*(a[1]*M(1,0) + b[1]*M(0,1)) + 
                  3*vo[1]*(a[1]^2*M(2,0) + 2*a[1]*b[1]*M(1,1) + b[1]^2*M(0,2)) + 
                  a[1]^3*M(3,0) + 3*a[1]^2*b[1]*M(2,1) + 3*a[1]*b[1]^2*M(1,2) + b[1]^3*M(0,3)
    elseif alpha == 1 && beta == 2 && gamma == 0 #secondMoment
        area_tt = vo[1]*vo[2]^2*M(0,0) + 
                  2*vo[1]*vo[2]*(a[2]*M(1,0) + b[2]*M(0,1)) + 
                  vo[1]*(a[2]^2*M(2,0) + 2*a[2]*b[2]*M(1,1) + b[2]^2*M(0,2)) + 
                  vo[2]^2*(a[1]*M(1,0) + b[1]*M(0,1)) + 
                  2*vo[2]*(a[1]*(a[2]*M(2,0) + b[2]*M(1,1)) + b[1]*(a[2]*M(1,1)+b[2]*M(0,2))) + 
                  a[1]*(a[2]^2*M(3,0) + 2*a[2]*b[2]*M(2,1) + b[2]^2*M(1,2)) + 
                  b[1]*(a[2]^2*M(2,1) + 2*a[2]*b[2]*M(1,2) + b[2]^2*M(0,3))
    elseif alpha == 1 && beta == 0 && gamma == 2 #secondMoment
        area_tt = vo[1]*vo[3]^2*M(0,0) + 
                  2*vo[1]*vo[3]*(a[3]*M(1,0) + b[3]*M(0,1)) + 
                  vo[1]*(a[3]^2*M(2,0) + 2*a[3]*b[3]*M(1,1) + b[3]^2*M(0,2)) + 
                  vo[3]^2*(a[1]*M(1,0) + b[1]*M(0,1)) + 
                  2*vo[3]*(a[1]*(a[3]*M(2,0) + b[3]*M(1,1)) + b[1]*(a[3]*M(1,1) + b[3]*M(0,2))) + 
                  a[1]*(a[3]^2*M(3,0) + 2*a[3]*b[3]*M(2,1) + b[3]^2*M(1,2)) + 
                  b[1]*(a[3]^2*M(2,1) + 2*a[3]*b[3]*M(1,2) + b[3]^2*M(0,3))
    elseif alpha == 1 && beta == 1 && gamma == 1 #inertiaProduct
        area_tt = vo[1]*vo[2]*vo[3]*M(0,0) + 
                  vo[1]*vo[2]*(a[3]*M(1,0) + b[3]*M(0,1)) + 
                  vo[1]*vo[3]*(a[2]*M(1,0) + b[2]*M(0,1)) + 
                  vo[1]*(a[2]*(a[3]*M(2,0) + b[3]*M(1,1)) + b[2]*(a[3]*M(1,1) + b[3]*M(0,2))) + 
                  vo[2]*vo[3]*(a[1]*M(1,0) + b[1]*M(0,1)) + 
                  vo[2]*(a[1]*(a[3]*M(2,0) + b[3]*M(1,1)) + b[1]*(a[3]*M(1,1) + b[3]*M(0,2))) + 
                  vo[3]*(a[1]*(a[2]*M(2,0) + b[2]*M(1,1)) + b[1]*(a[2]*M(1,1) + b[2]*M(0,2))) +
                  a[1]*(a[2]*(a[3]*M(3,0) + b[3]*M(2,1)) + b[2]*(a[3]*M(2,1) + b[3]*M(1,2))) + 
                  b[1]*(a[2]*(a[3]*M(2,1) + b[3]*M(1,2)) + b[2]*(a[3]*M(1,2) + b[3]*M(3,0)))
    elseif alpha == 2 && beta == 0 && gamma == 1 #inertiaProduct
        area_tt = vo[1]^2*vo[3]*M(0,0) +
                  vo[1]^2*(a[3]*M(1,0) + b[3]*M(0,1)) +
                  2*vo[1]*vo[3]*(a[1]*M(1,0) + b[1]*M(0,1)) +
                  2*vo[1]*(a[1]*(a[3]*M(2,0) + b[3]*M(1,1)) + b[1]*(a[3]*M(1,1) + b[3]*M(0,2))) +
                  vo[3]*(a[1]^2*M(2,0) + 2*a[1]*b[1]*M(1,1) + b[1]^2*M(0,2)) +
                  a[1]^2*(a[3]*M(3,0) + b[3]*M(2,1)) +
                  2*a[1]*b[1]*(a[3]*M(2,1) + b[3]*M(1,2)) +
                  b[1]^2*(a[3]*M(1,2) + b[3]*M(0,3))
    elseif alpha == 2 && beta == 1 && gamma == 0 #inertiaProduct
        area_tt = vo[1]^2*vo[2]*M(0,0) +
                  vo[1]^2*(a[2]*M(1,0) + b[2]*M(0,1)) +
                  2*vo[1]*vo[2]*(a[1]*M(1,0) + b[1]*M(0,1)) +
                  2*vo[1]*(a[1]*(a[2]*M(2,0) + b[2]*M(1,1)) + b[1]*(a[2]*M(1,1) + b[2]*M(0,2))) +
                  vo[2]*(a[1]^2*M(2,0) + 2*a[1]*b[1]*M(1,1) + b[1]^2*M(0,2)) +
                  a[1]^2*(a[2]*M(3,0) + b[2]*M(2,1)) +
                  2*a[1]*b[1]*(a[2]*M(2,1) + b[2]*M(1,2)) +
                  b[1]^2*(a[2]*M(1,2) + b[2]*M(0,3))
    else
        area_tt = s1(a, b, alpha, beta, gamma, vo)
    end
    if signedInt == true
        return area_tt * norm(c) * sign(c[3])
    else
        return area_tt * norm(c)
    end
end

Prendiamo i tempi di calcolo del volume, utilizzando sempre l'oggetto *Stanford Bunny*.

In [ ]:
V, EV, FV = obj2lar("stanford-bunny.obj")
EV = EV[1]
FV = FV[1]
VV = [[k] for k=1:size(V,2)]
model = (V, (VV, EV, FV))
P = V, FV

In [ ]:
@benchmark volume(P)

## Parallelizzazione

In questa sezione sono riportate le modifiche fatte al codice per quanto riguarda la sua parallelizzazione.

In [ ]:
using Base.Threads

In [ ]:
function II(P::LAR, alpha::Int, beta::Int, gamma::Int, signedInt=false)::Float64
    V, FV = P
    partialSum = zeros(nthreads())
    @threads for i=1:length(FV)
        tau = hcat([V[:,v] for v in FV[i]]...)
        if size(tau,2) == 3
            term = TT(tau, alpha, beta, gamma, signedInt)
            if signedInt
                @inbounds partialSum[threadid()] += term
            else
                @inbounds partialSum[threadid()] += abs(term)
            end
        elseif size(tau,2) > 3
            println("ERROR: FV[$(i)] is not a triangle")
        else
            println("ERROR: FV[$(i)] is degenerate")
        end
    end    
    return sum(partialSum)
end

In [ ]:
function III(P::LAR, alpha::Int, beta::Int, gamma::Int, signedInt::Bool=false)::Float64
    V, FV = P
    partialSum = zeros(nthreads())
    @threads for i=1:length(FV)
        tau = hcat([V[:,v] for v in FV[i]]...)
        vo,va,vb = tau[:,1],tau[:,2],tau[:,3]
        a = va - vo
        b = vb - vo
        c = cross(a,b)
        term = c[1]/norm(c) * TT(tau, alpha+1, beta, gamma, signedInt)
        if signedInt
            @inbounds partialSum[threadid()] += term
        else
            @inbounds partialSum[threadid()] += abs(term)
        end
    end
    return sum(partialSum)/(alpha + 1)
end

In [ ]:
function firstMoment(P::LAR)::Array{Float64,1}
    out = zeros(3)
    @async begin
        out[1] = III(P, 1, 0, 0)
        out[2] = III(P, 0, 1, 0)
        out[3] = III(P, 0, 0, 1)
    end
    return fetch(out)
end

In [ ]:
function secondMoment(P::LAR)::Array{Float64,1}
    out = zeros(3)
    @async begin
        out[1] = III(P, 2, 0, 0)
        out[2] = III(P, 0, 2, 0)
        out[3] = III(P, 0, 0, 2)
    end
    return fetch(out)
end

In [ ]:
function inertiaProduct(P::LAR)::Array{Float64,1}
    out = zeros(3)
    @async begin
        out[1] = III(P, 0, 1, 1)
        out[2] = III(P, 1, 0, 1)
        out[3] = III(P, 1, 1, 0)
    end
    return fetch(out)
end

Prendiamo i tempi di calcolo del volume, utilizzando sempre l'oggetto *Stanford Bunny*.

In [ ]:
V, EV, FV = obj2lar("stanford-bunny.obj")
EV = EV[1]
FV = FV[1]
VV = [[k] for k=1:size(V,2)]
model = (V, (VV, EV, FV))
P = V, FV

In [ ]:
@benchmark volume(P)

La macro `@benchmark` portava l'esecuzione delle funzioni `firstMoment`, `secondMoment` e `inertiaProduct` alla non terminazione quando al loro interno veniva utilizzata la macro `@async`, quindi si è deciso di utilizzare la macro `@time` anche se meno precisa ai fini del testing.

In [ ]:
@time firstMoment(P)